In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import progressbar
from sklearn.model_selection import train_test_split,KFold
from all_function_class_for_real_complex_final_bias import convolution
from all_function_class_for_real_complex_final_bias import flatten, max_pool_real, max_pool_complex, relu_real, cartReLu, dense
from all_function_class_for_real_complex_final_bias import layer_init_real, layer_init_complex
from all_function_class_for_real_complex_final_bias import train, test, max_pool_real_new
drive.mount('/content/drive')
tf.executing_eagerly()
tf.config.set_soft_device_placement(True)

from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [ ]:
# Read data
# Please change the "pathtodata" in the following directory to the path you put the data into.
data_1 = scipy.io.loadmat('/pathtodata/data_1_fft.mat')
data_2 = scipy.io.loadmat('/pathtodata/data_2_fft.mat')
data_3 = scipy.io.loadmat('/pathtodata/data_3_fft.mat')
data_4 = scipy.io.loadmat('/pathtodata/data_4_fft.mat')
data_5 = scipy.io.loadmat('/pathtodata/data_5_fft.mat')

data_1_fft = data_1['data_1']
data_2_fft = data_2['data_2']
data_3_fft = data_3['data_3']
data_4_fft = data_4['data_4']
data_5_fft = data_5['data_5']

del data_1
del data_2
del data_3
del data_4
del data_5

X_all = np.concatenate(
                      (
                       tf.reshape(np.complex64(data_1_fft),[-1,1,128,1]), 
                       tf.reshape(np.complex64(data_2_fft),[-1,1,128,1]),
                       tf.reshape(np.complex64(data_3_fft),[-1,1,128,1]), 
                       tf.reshape(np.complex64(data_4_fft),[-1,1,128,1]),
                       tf.reshape(np.complex64(data_5_fft),[-1,1,128,1])
                       ),
                       axis = 0)

y_all = np.uint8(np.concatenate((
                                np.zeros(data_1_fft.shape[0]),
                                np.ones(data_2_fft.shape[0]),
                                2*np.ones(data_3_fft.shape[0]),
                                3*np.ones(data_4_fft.shape[0]),
                                4*np.ones(data_5_fft.shape[0])
                                 )))


del data_1_fft
del data_2_fft
del data_3_fft
del data_4_fft
del data_5_fft


In [ ]:
def initialization(num_class):

    # Initialization

    std = 0.01
            
    train_vars = [layer_init_complex([1, 5, 1, 8], 1),
                  layer_init_real([1, 5, 8, 16], std),
                  layer_init_real([1, 5, 16, 32], std),
                  layer_init_real([1152, 128], std),
                  layer_init_real([128, 32], std),
                  layer_init_real([32, num_class], std)]

    vars_name = {}
    for i in range(len(train_vars)):
        ker_name = 'ker_{}'.format(i)
        bias_name = 'bias_{}'.format(i)
        vars_name[ker_name] = train_vars[i].ker
        vars_name[bias_name] = train_vars[i].bias

    return {'train_vars': train_vars, 'vars_name':vars_name}

In [ ]:
# Define tf_function
@tf.function
def layer_comb(X, Y, arguments, my_vars):
    arg_iter = iter(range(len(arguments)))
    with tf.GradientTape(persistent=True) as t:
        ########################################################################
        ind = next(arg_iter)
        X = convolution(X, arguments[ind].ker, arguments[ind].bias, padding = 0, stride=1)
        X = tf.math.abs(X)
        ########################################################################
        ind = next(arg_iter)
        X = convolution(X, arguments[ind].ker, arguments[ind].bias, padding=0, stride=1)
        X = relu_real(X)
        X = max_pool_real_new(X, pool_h=1, pool_w=3, padding=0, stride=3)
        ########################################################################
        ind = next(arg_iter)
        X = convolution(X, arguments[ind].ker, arguments[ind].bias, padding=0, stride=1)
        X = relu_real(X)
        ########################################################################
        ind = next(arg_iter)
        X = dense(X, arguments[ind].ker, arguments[ind].bias)
        X = relu_real(X)
        ########################################################################
        ind = next(arg_iter)
        X = dense(X, arguments[ind].ker, arguments[ind].bias)
        X = relu_real(X)
        ########################################################################
        ind = next(arg_iter)
        X = dense(X, arguments[ind].ker, arguments[ind].bias)

        X_softmax = tf.nn.softmax(X)

        loss = -tf.reduce_sum(Y*tf.math.log(tf.clip_by_value(X_softmax,1e-10,1.0)))/Y.shape[0]

    grad = t.gradient(loss, my_vars)
    X_label = tf.cast(tf.argmax(X_softmax, axis=1), dtype=tf.float32)

    return {'loss': loss, 'grad': grad, 'X_label': X_label}

In [ ]:
# Cross Validation
num_class = len(np.unique(y_all))
repeat_time = 20
acc_test = np.zeros([repeat_time, 5])
# Try different seeds for split
for seed in range(repeat_time):
    kf = KFold(n_splits=5,  random_state=5, shuffle=True)
    split_5 = list(kf.split(X_all))
    conf_mat = np.zeros([5,5])
    # 5-fold Cross Validation
    for indx in range(5):
        # print(indx)
        # Record start time
        start_time = time.time()

        x_train = X_all[split_5[indx][0],:,:,:]
        y_train = tf.one_hot(y_all[split_5[indx][0]], num_class)
        x_test = X_all[split_5[indx][1],:,:,:]
        y_test = tf.one_hot(y_all[split_5[indx][1]], num_class)

        # Parameters initialization
        para_init = initialization(num_class)

        # For train set
        train_main_result = train(layer_comb, x_train, y_train, para_init['train_vars'], 
                                  para_init['vars_name'], epoch = 50, train_batch_size=32,
                                  decay=True, decay_rate=0.95)
        # For test set
        test_main_result = test(layer_comb, x_test, y_test, para_init['train_vars'], 
                                para_init['vars_name'], test_batch_size = 128)

        # Show the total time consuming for one split 
        print("--- %s seconds ---" % (time.time() - start_time))

        print(test_main_result['acc'])
        # print(test_main_result['conf_mat'])
        # print(test_main_result['conf_mat']/np.sum(test_main_result['conf_mat'], axis=1, keepdims=True))
        acc_test[seed, indx] = test_main_result['acc']
        conf_mat += test_main_result['conf_mat']
    print(acc_test.mean(axis=1))
    print(acc_test.std(axis=1))
    print(conf_mat/np.sum(conf_mat, axis=1, keepdims=True))
print(acc_test.mean())
print(acc_test.std())

In [ ]:
acc_test.mean()

In [ ]:
acc_test.std()